In [ ]:
import os

api_key = "..."
os.environ["OPENAI_API_KEY"] = api_key

os.environ.get("OPENAI_API_KEY")

# Load document

In [1]:
# text document
from langchain_community.document_loaders import TextLoader

loader = TextLoader("../dataset/llamaindex_data/openai.txt")
docs = loader.load()

RuntimeError: Error loading ../dataset/llamaindex_data/openai.txt

In [ ]:
# pdf document
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../dataset/llamaindex_data/Mistral AI - Wikipedia.pdf")
docs = loader.load()

In [ ]:
# directory 내 문서 모두 불러오기
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("../dataset/llamaindex_data", glob="*", show_progress=True)
docs = loader.load()

# Chunking 

In [3]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False
)

In [4]:
documents = text_splitter.split_documents(docs)

NameError: name 'docs' is not defined

# Recursive Text Splitter

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " "],
    chunk_size=500,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False
)

In [ ]:
documents = text_splitter.split_documents(docs)

In [6]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

text_splitter = SemanticChunker(OpenAIEmbeddings(api_key=api_key))
documents = text_splitter.split_documents(docs)

NameError: name 'api_key' is not defined

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

# tiktoken -> OpenAI 가 개발
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
documents = text_splitter.split_documents(docs)

# Embedding

In [8]:
embed_model = OpenAIEmbeddings(api_key=api_key, model="text-embedding-3-small")
emb = embed_model.embed_query("What is Mistral AI?")

NameError: name 'api_key' is not defined

In [9]:
emb = embed_model.embed_documents(["What is Mistral AI?", "ML"])

NameError: name 'embed_model' is not defined

# VectorStore

In [10]:
from langchain_community.vectorstores import Chroma

vector_index = Chroma.from_documents(documents, embed_model)

NameError: name 'documents' is not defined

In [11]:
retrieved = vector_index.similarity_search("What is Mistral AI?")

NameError: name 'vector_index' is not defined

In [12]:
from langchain_community.vectorstores import FAISS

vector_index = FAISS.from_documents(documents, embed_model)

NameError: name 'documents' is not defined

In [ ]:
retrieved = vector_index.similarity_search("What is Mistral AI?")
retrieved[0].page_content

# Retriever

In [ ]:
query = "OpenAI의 sora 모델에 대해 알려줘"

retriever = vector_index.as_retriever(search_type="mmr")
retriever.get_relevant_documents(query)

In [13]:
retriever = vector_index.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold: 0.2"})
retriever.get_relevant_documents(query)

NameError: name 'vector_index' is not defined

In [14]:
retriever = vector_index.as_retriever(search_kwargs={"k": 3})
retriever.get_relevant_documents(query)

NameError: name 'vector_index' is not defined

In [15]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retievers.multi_query").setLevel(logging.INFO)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

retriever_mult = MultiQueryRetriever.from_llm(retriever=vector_index.as_retriever(), llm=llm)

In [ ]:
retriever_mult.get_relevant_documents(query="OpenAI의 sora 모델에 대해 알려줘")m

In [16]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

store = InMemoryStore()

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

vector_index = Chroma(collection_name="split_parents", embedding_function=embed_model)


NameError: name 'embed_model' is not defined

In [17]:
retriever = ParentDocumentRetriever(
    vectorstore=vector_index,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

NameError: name 'vector_index' is not defined

In [18]:
retriever.get_relevant_documents(query="OpenAI의 sora 모델에 대해 알려줘") # 아무것도 안나옴 다큐먼트 없음 

NameError: name 'retriever' is not defined

In [19]:
retriever.add_documents(docs)

NameError: name 'retriever' is not defined

In [20]:
retriever.get_relevant_documents(query="OpenAI의 sora 모델에 대해 알려줘")

NameError: name 'retriever' is not defined

# Generator

In [21]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

NameError: name 'ChatOpenAI' is not defined

In [22]:
llm.invoke("OpenAI 의 sora 모델에 대해 알려줘")

NameError: name 'llm' is not defined

# Chain

In [23]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me something")

chain = prompt | llm

NameError: name 'llm' is not defined

In [24]:
chain.invoke({})

NameError: name 'chain' is not defined

In [25]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()

NameError: name 'llm' is not defined

In [26]:
chain.invoke({})

NameError: name 'chain' is not defined

In [27]:
prompt = ChatPromptTemplate.from_template("Tell me something about {topic}")
chain = prompt | llm | StrOutputParser()

NameError: name 'llm' is not defined

In [28]:
chain.invoke({"topic": "sora, developed by OpenAI"})

NameError: name 'chain' is not defined

In [29]:
prompt_formatted = prompt.invoke({"topic": "sora, developed by OpenAI"})
prompt_formatted

ChatPromptValue(messages=[HumanMessage(content='Tell me something about sora, developed by OpenAI', additional_kwargs={}, response_metadata={})])

In [ ]:
model_output = llm.invoke(prompt_formatted)
model_output

In [ ]:
parser = StrOutputParser()
parser.invoke(model_output)

In [ ]:
(prompt|llm|StrOutputParser()).invoke({"topic": "sora, developed by OpenAI"})

# RAG Chain

In [30]:
retrieved_docs = retriever.invoke("Tell me something about sora, developed by OpenAI")
retrieved_docs

NameError: name 'retriever' is not defined

In [ ]:
def merge_docs(retrieved_docs):
    return "\n\n".join([doc.page_content for doc in retrieved_docs])

In [31]:
(retriever | merge_docs | llm | parser).invoke("Tell me something about sora, developed by OpenAI")

NameError: name 'retriever' is not defined

In [ ]:
from langchain_core.runnables import RunnableParallel

chain_parallel = RunnableParallel({"context": retriever, "llm": llm})

In [ ]:
chain_parallel.invoke("Tell me something about sora, developed by OpenAI")

In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain_parallel = RunnableParallel({"context": retriever, "query": RunnablePassthrough()})

In [32]:
chain_parallel.invoke("Tell me something about sora, developed by OpenAI")

NameError: name 'chain_parallel' is not defined

In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate

template = """
Utilize the context given below, answer the question.
[context]
{context}

question: {query}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = RunnableParallel({"context": retriever, "query": RunnablePassthrough()})\
    | prompt\
    | llm \
    | StrOutputParser()

In [ ]:
chain.invoke("Tell me something about sora, developed by OpenAI")

In [ ]:
def foo(anything):
    return "bar"

In [34]:
chain = RunnableParallel(context=retriever, query=RunnablePassthrough(), asdf=foo) \
        | prompt \
        | llm \
        | StrOutputParser()

NameError: name 'RunnableParallel' is not defined

In [35]:
chain = {"context": retriever, "query": RunnablePassthrough()} \
        | prompt \
        | llm \
        | StrOutputParser()

NameError: name 'retriever' is not defined

In [36]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt.messages

/Users/harim/PycharmProjects/pythonProject/.venv/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [ ]:
chain = RunnableParallel({"context": retriever, "query": RunnablePassthrough()}) \
        | prompt \
        | llm \
        | StrOutputParser()

# Putting it all together

In [37]:
loader = DirectoryLoader("../dataset/llamaindex_data", glob="*", show_progress=True)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " "],
    chunk_size=500,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False
)

documents = text_splitter.split_documents(docs)

NameError: name 'DirectoryLoader' is not defined

In [38]:
embed_model = OpenAIEmbeddings(api_key=api_key, model="text-embedding-3-small")

NameError: name 'api_key' is not defined

In [39]:
vector_index = FAISS.from_documents(documents, embed_model)

NameError: name 'documents' is not defined

In [40]:
retriever = vector_index.as_retriever(search_type="mmr")

NameError: name 'vector_index' is not defined

In [41]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

NameError: name 'ChatOpenAI' is not defined

In [42]:
prompt = hub.pull("rlm/rag-prompt")

/Users/harim/PycharmProjects/pythonProject/.venv/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [43]:
def merge_docs(retrieved_docs):
    return "\n\n".join([doc.page_content for doc in retrieved_docs])

In [44]:
chain = RunnableParallel(context=retriever | merge_docs, question=RunnablePassthrough())\
    | prompt \
    | llm \
    | StrOutputParser()

NameError: name 'RunnableParallel' is not defined

In [45]:
chain.invoke("Tell me something about sora, developed by OpenAI")

NameError: name 'chain' is not defined